<center><h1>Testing Notebook</h1></center>


## Loading Entities:

 - Power System
 - Ant Colony

In [ ]:
from acopoweropt import system, colony

PSystem = system.PowerSystem(name='s10')
Colony = colony.Colony(n_ants=5,phr_evp_rate=0.25)

## Testing Power System

In [ ]:
# Randomly select an operation
operation = PSystem.sample_operation()

# Solve system
solution = PSystem.solve(operation=operation)

In [ ]:
solution.get('Ft')

## Testing Colony

In [1]:
import pandas as pd
import numpy as np

from acopoweropt import colony, system



In [3]:
import pandas as pd
import numpy as np

from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s15')
Colony = colony.PowerColony(n_ants=10,
                            pheromone_evap_rate=0.25,
                            PowerSystem=PSystem)

print("Iteracao: {} - MinValue: {}".format(0, Colony.paths[0].distance.min()))

n_iter = 50

for i in range(n_iter):
    # Getting last taken paths
    paths = Colony.paths[i].copy()
    # Evaporate Pheromone
    Colony.pheromone = (1 - Colony.pheromone_evap_rate) * Colony.pheromone
   
    # Selecting Ants (80/20) to follow or not the pheromone paths
    fol_ants = paths.sample(frac=0.8, replace=False, random_state=1)
    rnd_ants = paths.drop(fol_ants.index)

    l = []
    for ant in fol_ants.index:
        operative_zones = Colony.choose_path()
        chosen_path = ",".join([str(int) for int in operative_zones])
        existing_path = paths.query("path == '{}'".format(chosen_path)) 
        
        ## Check if chosen path was already calculated:
        if existing_path.values.size != 0:
            l.append(existing_path)
        else:
            operation = PSystem.get_operation(operative_zones=operative_zones)
            solution = pd.DataFrame([Colony.seek_food(ant=ant, operation=operation, PowerSystem=PSystem)]).set_index('ant')
            l.append(solution)
    
    for ant in rnd_ants.index:
        operation = PSystem.sample_operation()
        solution = pd.DataFrame([Colony.seek_food(ant=ant, operation=operation, PowerSystem=PSystem)]).set_index('ant')
        l.append(solution)

    new_paths = pd.concat(l).sort_index()
    Colony.paths.update({i+1: new_paths})
    
    
    # Updating Pheromone
    new_pheromone = Colony.update_pheromone(paths=new_paths)
    Colony.pheromone_history.update({i+1: new_pheromone})
    
    print("Iteracao: {} - MinValue: {}".format(i+1, paths.distance.min()))

Iteracao: 0 - MinValue: 32583.629747776522
Iteracao: 1 - MinValue: 32583.629747776522
Iteracao: 2 - MinValue: 32614.570709239342
Iteracao: 3 - MinValue: 32640.6879144951
Iteracao: 4 - MinValue: 32552.04791396032
Iteracao: 5 - MinValue: 32583.310213033616
Iteracao: 6 - MinValue: 32552.04791396032
Iteracao: 7 - MinValue: 32598.523380515573
Iteracao: 8 - MinValue: 32561.557341875046
Iteracao: 9 - MinValue: 32560.359066619472
Iteracao: 10 - MinValue: 32581.833560660405
Iteracao: 11 - MinValue: 32561.557341875046
Iteracao: 12 - MinValue: 32561.557341875046
Iteracao: 13 - MinValue: 32571.39808200774
Iteracao: 14 - MinValue: 32598.523380515573
Iteracao: 15 - MinValue: 32572.569313792294
Iteracao: 16 - MinValue: 32688.619792407902
Iteracao: 17 - MinValue: 32547.487810882907
Iteracao: 18 - MinValue: 32550.63564329305
Iteracao: 19 - MinValue: 32597.91399600845
Iteracao: 20 - MinValue: 32614.570709239342
Iteracao: 21 - MinValue: 32646.3004203997
Iteracao: 22 - MinValue: 32560.43673580717
Iteracao

In [4]:
operation

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
opz,,,,,,,,,,,,,,,
1,0.248129,0.001673,0.248129,0.248129,0.032641,0.207545,0.248129,0.248129,0.248129,0.248129,0.248129,0.001263,0.248129,0.248129,0.248129
2,0.000000,0.009239,0.000000,0.000000,0.073363,0.019755,0.000000,0.000000,0.000000,0.000000,0.000000,0.160111,0.000000,0.000000,0.000000
3,0.000000,0.220493,0.000000,0.000000,0.140394,0.000896,0.000000,0.000000,0.000000,0.000000,0.000000,0.086755,0.000000,0.000000,0.000000
4,0.000000,0.016723,0.000000,0.000000,0.001730,0.019932,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
optimal = [1,3,1,1,3,4,1,1,1,1,1,2,1,1,1]
test = [1,2,1,1,2,3,1,1,1,1,1,2,1,1,1]

operation = PSystem.get_operation(operative_zones=test)

PSystem.solve(operation=operation)

# pd.DataFrame([Colony.seek_food(ant=1, operation=operation, PowerSystem=PSystem)]).set_index('ant')

{'status': 'optimal',
 'Ft': 32702.108929564645,
 'operation':      opz       a      b         c Pmin Pmax          Pg           Fi
 tgu                                                                 
 1      1  671.03  10.07  0.000299  150  455  454.999922  4643.749964
 2      2  574.54  10.22  0.000183  225  305  304.999927     3134.123
 3      1  374.59    8.8  0.001126   20  130  129.999959  1163.030152
 4      1  374.59    8.8  0.001126   20  130  129.999959  1163.030152
 5      2  461.37   10.4  0.000205  200  260  259.999919  2717.857352
 6      3  630.14   10.1  0.000301  395  430  429.999920  4398.654374
 7      1   548.2   9.87  0.000364  135  465  464.999927  4668.255522
 8      1  227.09   11.5  0.000338   60  300   60.005308   691.278395
 9      1  173.72  11.21  0.000807   25  162  109.764614  1240.185081
 10     1  175.95  10.72  0.001203   20  160  159.999813  1745.995923
 11     1  186.86  11.21  0.003586   20   80   25.230441   285.119591
 12     2  230.27    9.9  0.

In [12]:
[[1 2 1 1 2 3 1 1 1 1 1 2 1 1 1], [1 3 1 1 3 1 1 1 1 1 1 3 1 1 1], [1 2 1 1 2 1 1 1 1 1 1 2 1 1 1], [1 4 1 1 3 3 1 1 1 1 1 2 1 1 1], [1 3 1 1 1 2 1 1 1 1 1 3 1 1 1]]

,path,status,distance,tau
ant,,,,
1,"1,2,1,1,4,2,1,1,1,1,1,1,1,1,1",optimal,32640.082022,0.000031
2,"1,3,1,1,2,3,1,1,1,1,1,1,1,1,1",optimal,32621.473431,0.000031
3,"1,1,1,1,1,4,1,1,1,1,1,2,1,1,1",optimal,32894.216281,0.000030
4,"1,4,1,1,4,1,1,1,1,1,1,2,1,1,1",optimal,32621.266691,0.000031
5,"1,2,1,1,2,2,1,1,1,1,1,2,1,1,1",optimal,32773.076734,0.000031


In [ ]:
.size == 0

In [50]:
paths.query("path == '{}'".format(chosen_path)).values.size == 0

True

In [47]:
Colony.paths

{0:                               path   status      distance       tau
 ant                                                                
 1    1,2,1,1,2,4,1,1,1,1,1,2,1,1,1  optimal  32672.124337  0.000031
 2    1,1,1,1,4,4,1,1,1,1,1,3,1,1,1  optimal  32635.669146  0.000031
 3    1,3,1,1,4,2,1,1,1,1,1,1,1,1,1  optimal  32575.017052  0.000031
 4    1,4,1,1,2,4,1,1,1,1,1,2,1,1,1  optimal  32558.357929  0.000031
 5    1,3,1,1,1,3,1,1,1,1,1,1,1,1,1  optimal  32688.619792  0.000031}

In [209]:
[Colony.pheromone.sample(n=1, weights=Colony.pheromone[tgu], axis=0).index[0] for tgu in Colony.pheromone.columns]

[1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]

[1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]

In [161]:
Colony.pheromone.sample(n=1, weights=Colony.pheromone[2], axis=0).index[0]

4

In [165]:
Colony.pheromone

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
opz,,,,,,,,,,,,,,,
1,0.3066,0.0000,0.3066,0.3066,0.0000,0.0612,0.3066,0.3066,0.3066,0.3066,0.3066,0.0612,0.3066,0.3066,0.3066
2,0.0000,0.0000,0.0000,0.0000,0.1840,0.0612,0.0000,0.0000,0.0000,0.0000,0.0000,0.1226,0.0000,0.0000,0.0000
3,0.0000,0.1226,0.0000,0.0000,0.1226,0.1842,0.0000,0.0000,0.0000,0.0000,0.0000,0.1228,0.0000,0.0000,0.0000
4,0.0000,0.1840,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [66]:
import random


random.choices(population=Colony.pheromone[1], weights=[Colony.pheromone[1][i+1]/Colony.pheromone[1].sum() for i,n in enumerate(Colony.pheromone[1])],k=1)

[1]

In [75]:
[Colony.pheromone[1][i+1]/Colony.pheromone[1].sum() for i,n in enumerate(Colony.pheromone[1])]

[1.0, 0.0, 0.0, 0.0]

In [69]:
Colony.pheromone[1].sum()

0.3066